# LIBRARY

In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time

# DATA SOURCE

In [2]:
data_source = "https://www.instagram.com/kellykieuu/"

path = "C:/Users/JamesDuong/.wdm/drivers/chromedriver/win32/87.0.4280.88/chromedriver.exe"
driver = webdriver.Chrome(path)
#driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get(data_source)

# EXTRACT DATA

In [3]:
# GET OVERVIEW
user_name = driver.find_element_by_class_name('-vDIg').find_element_by_tag_name('h1').text
bio = driver.find_element_by_class_name('-vDIg').find_element_by_tag_name('span').text
url_in_Bio = driver.find_element_by_class_name('-vDIg').find_element_by_tag_name('a').text

posts_num, followers_num, following_num = driver.find_elements_by_class_name('-nal3 ')
posts_num = posts_num.find_element_by_tag_name('span').text
followers_num = followers_num.find_element_by_tag_name('span').text
following_num = following_num.find_element_by_tag_name('span').text


# Scroll page
lenOfPage = driver.execute_script('window.scrollTo(0, document.body.scrollHeight); var lenOfPage=document.body.scrollHeight; return lenOfPage;')
match=False
while(match==False):
        lastCount = lenOfPage
        time.sleep(1)
        lenOfPage = driver.execute_script('window.scrollTo(0, document.body.scrollHeight); var lenOfPage=document.body.scrollHeight; return lenOfPage;')
        if lastCount == lenOfPage:
            match=True


# GET ALL POSTS' URL
posts = []
links = driver.find_elements_by_tag_name('a')
for link in links:
    post = link.get_attribute('href')
    if '/p/' in post:
        posts.append(post)

# CLEAN DATA

In [ ]:
Dict_image = {'photo_link': [],
          'likes': [],
          'comments': []}

Dict_video = {'video_link': [],
          'likes': [],
          'comments': []}


# Go through each post url, extract & clean desirable data 
i = 0
for post in posts:
    i += 1
    driver.get(post)
    shortcode = driver.current_url.split('/')[-2]
    
    # Determine a post type, either photo or video 
    Type = driver.find_element_by_xpath('//meta[@property="og:type"]').get_attribute('content')
    if Type =='instapp:photo':
        photo_link = driver.find_element_by_xpath('//meta[@property="og:image"]').get_attribute('content')

        Dict_image['photo_link'].append(photo_link)
    else:
        video_link = driver.find_element_by_xpath('//meta[@property="og:video"]').get_attribute('content')

        Dict_video['video_link'].append(video_link)
    
    # Extract the number of LIKES, COMMENTS 
    try:
        like_comment = driver.find_element_by_xpath('//meta[@property="og:description"]').get_attribute('content')
        likes, comments = like_comment.split('-')[0].split(',')
        # clean
        likes = likes.replace(' Likes', '')
        comments = comments.replace('Comments', '').replace(' ', '')
    except:
        likes = driver.find_element_by_class_name('Nm9Fw').find_element_by_tag_name('span').text
        comments = -1
    
    try:
        likes = int(likes)
        comments = int(comments)
    except:
        print(likes, comments)
        pass
    
    Dict_image['likes'].append(likes)
    Dict_image['comments'].append(comments)
    
    #print(post, likes, comments)
    
    if i == 10:
        break

# BUSINESS LOGIC LAYER

In [ ]:
# Store Data in dataframe
df_ig = pd.DataFrame(data=Dict_image)

# Change data type of features
df_ig['likes'] = df_ig['likes'].astype(int)
df_ig['comments'] = df_ig['comments'].astype(int)
df_ig.dtypes

# Sorted top 5 most liked post on Instagram
top_5_most_liked_ig_post = df_ig.sort_values('likes', ascending=False)[:5]

# Sorted top 5 most commented post on Instagram
top_5_most_commented_ig_post = df_ig.sort_values('comments', ascending=False)[:5]

# SAVE TO EXCEL

In [ ]:
from openpyxl import Workbook

import openpyxl

book = openpyxl.load_workbook('pandas_positioning.xlsx')

sheet = book.active
# a1 = sheet['A1']
# a2 = sheet['A2']
# a3 = sheet.cell(row=3, column=1)

# print(a1.value)
# print(a2.value) 
# print(a3.value)

#a3 = sheet.cell(row=3, column=2)
#print(a3.value)

#print(book.get_sheet_names())

# Open the sheet "IG" in excel file
sheet_ig = book.get_sheet_by_name("IG")

In [ ]:
# IG URL
sheet['B3'] = data_source

# Bio
sheet['C3'] = bio

# URL in Bio
sheet['D3'] = url_in_Bio

# TOTAL FOLLOWERS
sheet['B11'] = followers_num
# TOTAL POSTS
sheet['B15'] = posts_num

# TOP 5 LIKED POSTS
sheet['B19'] = top_5_most_liked_ig_post.iloc[0]['likes']
sheet['C19'] = top_5_most_liked_ig_post.iloc[1]['likes']
sheet['D19'] = top_5_most_liked_ig_post.iloc[2]['likes']
sheet['E19'] = top_5_most_liked_ig_post.iloc[3]['likes']

# Source: http://zetcode.com/python/openpyxl/
book.save("pandas_positioning.xlsx")